In [1]:
import numpy as np
import pandas as pd
import pylab as pl
import os
from IPython.display import clear_output

In [2]:
# from kaggle.api.kaggle_api_extended import KaggleApi
# import json
  
# with open('kaggle.json') as f:
#     data = json.load(f)

# os.environ['KAGGLE_USERNAME'] = data["username"]
# os.environ['KAGGLE_KEY'] = data["key"]

# api = KaggleApi()
# api.authenticate()

In [3]:
# api.dataset_download_files('rmisra/news-category-dataset', path="./data", unzip=True)

In [4]:
data = pd.read_json("data/News_Category_Dataset_v2.json", lines=True)

# Naive Bayes

In [46]:
data.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [47]:
data["full_text"] = data["headline"] + " " + data["short_description"]

In [48]:
X, y = data["full_text"].to_numpy(), data["category"].to_numpy()

In [50]:
import regex as re
from nltk import SnowballStemmer
import string

stemmer = SnowballStemmer('english')

numbers = re.compile('[0-9]')
punctuation = re.compile(f'[{string.punctuation.replace("$", "")}]')
hashtags = re.compile('#\w+')
for i in range(len(X)):
    # Remove hashtags
    X[i] = hashtags.sub('$hashtag', X[i])
    # Remove numbers
    X[i] = numbers.sub("0", X[i])
    # Remove punctuation
    X[i] = punctuation.sub("", X[i])
    # Lowercase
    X[i] = X[i].lower()
    # Stemming
    X[i] = stemmer.stem(X[i])


In [28]:
# join collocations
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
from nltk.corpus import stopwords

stopwords_ = set(stopwords.words('english'))

words = [word.lower() for document in X for word in document.split() 
         if len(word) > 2 and word not in stopwords_]
         
bgm = BigramAssocMeasures()
score = bgm.mi_like
finder = BigramCollocationFinder.from_words(words)
finder.apply_freq_filter(5)
collocations = [[bigram, pmi] for bigram, pmi in finder.score_ngrams(score)]
collocations = pd.DataFrame(collocations, columns=['bigram', 'pmi'])

In [29]:
# join collocations
def apply_collocations(sentence, set_colloc):
    for b1,b2 in set_colloc:
        sentence = sentence.replace("%s %s" % (b1 ,b2), "%s_%s" % (b1 ,b2))
    return sentence

set_collocations = collocations.sort_values(by='pmi', ascending=False).head(500)["bigram"].to_list()
for i in range(len(X)):
    X[i] = apply_collocations(X[i], set_collocations)
    # track progress
    if i % 5000 == 0:
        clear_output(wait=True)
        print(f"{i}/{len(X)}")

200000/200853


In [49]:
# Encode y labels
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(y)

In [45]:
# Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [47]:
# Tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(lowercase=True, min_df=0.0001, max_df=0.7, sublinear_tf=True, strip_accents="unicode", ngram_range=(1,2))

In [40]:
# Count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(lowercase=True, min_df=0.0001, max_df=0.7, strip_accents="unicode", ngram_range=(1,2), stop_words="english")

In [ ]:
# logreg
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=4, max_iter=1000, solver="saga", verbose=1, tol=0.1)

In [41]:
# naive bayes
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

In [46]:
# Pipeline
from sklearn.pipeline import Pipeline
pipe = Pipeline([('vectorizer', count_vectorizer), ('model', mnb)])
pipe.fit(X_train, y_train)

pipe.score(X_test, y_test)

0.5830076423290433

# CNN

In [5]:
import torch
from torch import nn
from torch.nn import functional as F
from CNN import *
from tokenizer import *

In [10]:
X = np.load("data/X_news_category_cleaned.npy", allow_pickle=True)
tokenizer = MyTokenizer(max_length=256)
tokenizer.fit(X)

In [8]:
data = pd.read_json("data/News_Category_Dataset_v2.json", lines=True)
y = data["category"].values
# Encode y labels
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(y)

y = torch.from_numpy(y).long()

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(torch.tensor(tokenizer.transform(X)), y, test_size=0.7)

In [12]:
import torch
from torch.utils.data import Dataset


class MatrixLoader(Dataset):
    """Simple dataloader given the data matrix X and response y
    batch is assumed to be the first dimension"""
    def __init__(self, X, y, classification=True):
        self.X = X
        self.y = y

        if classification:
            self.y = self.y.to(int)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [13]:
# Create datasets
train_set= MatrixLoader(X_train, y_train)
test_set = MatrixLoader(X_test, y_test)

# Create dataloaders
from torch.utils.data import DataLoader
train_loader = DataLoader(train_set, batch_size=128, shuffle=True)
test_loader = DataLoader(test_set, batch_size=256, shuffle=True)

In [14]:
from CNN import *
net = CNN(embedding_dim=64, n_resnets=5, blocks_per_resnet=1, kernel_size=3)

model = TextClassifier(tokenizer.get_vocab_size(), embedding_dim=64, out_dim=y.max()+1, net=net)

In [15]:
# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), 1e-2)
criterion = nn.CrossEntropyLoss()

losses = []
for epoch in range(10):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.cpu().item())
        print(batch_idx, loss.cpu().item())

        if batch_idx % 10 == 0:
            clear_output(True)
            pl.plot(losses)


    # Print eval loss
    model.eval()
    with torch.no_grad():
        test_loss = 0
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()

        test_loss /= batch_idx + 1
        print(f"Test loss: {test_loss}")

RuntimeError: cuDNN error: CUDNN_STATUS_EXECUTION_FAILED